# 1. 開発実習環境セットアップ

# 1.1. Pythonのパッケージのインストール

In [37]:
!pip install openai==0.27.8 
!pip install langchain==0.0.240
!pip install PyMuPDF==1.22.5
!pip install tiktoken==0.4.0
!pip install faiss-cpu==1.7.4

You should consider upgrading via the '/Users/shimac/.pyenv_x86_64/versions/3.10.4/bin/python3.10 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/shimac/.pyenv_x86_64/versions/3.10.4/bin/python3.10 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/shimac/.pyenv_x86_64/versions/3.10.4/bin/python3.10 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/shimac/.pyenv_x86_64/versions/3.10.4/bin/python3.10 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/shimac/.pyenv_x86_64/versions/3.10.4/bin/python3.10 -m pip install --upgrade pip' command.


## 1.2. Pythonのモジュールのインポート

In [38]:
import os
import json
from dotenv import load_dotenv

# OpenAI
import openai

# LangChain
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA

# Azure OpenAI Service
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import AzureChatOpenAI

## 1.3. 環境変数の設定他

.envファイルから環境変数をロードします。各環境にあわせて.env.templateを基に.evnファイルを作成してください。

In [39]:
load_dotenv()

True

In [40]:
print(os.getenv("OPENAI_API_VERSION"))

2023-05-15


Azure OpenAI ServiceのAPIを実行する際、デバッグレベルのログを出力するように設定します。

In [41]:
openai.log = "debug"

# 2. PDFファイルからベクトルデータベースのインデックス作成

## 2-1. PDFファイルを読み込み、分割

埋め込みモデルのinputの上限を超えず、情報源として役に立つサイズに分割します。

In [42]:
pdf_file_path = "./pdf/IBM_2021_ESG_Report.pdf"
loader = PyMuPDFLoader(pdf_file_path)

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    encoding_name="cl100k_base",
    separator="\n",
    chunk_size=2000,
    chunk_overlap=200,
    disallowed_special=(),
)
pages = loader.load_and_split(text_splitter=text_splitter)

PDFファイルを読み込んだ結果を確認してみます。

In [43]:
type_of_page = type(pages[0])
page_count = len(pages)

print("pageの型:", type_of_page)
print("ページ数:", page_count)

pageの型: <class 'langchain.schema.document.Document'>
ページ数: 65


1ページ分のデータを参照してみます。

In [44]:
page = pages[11]
formatted_data = json.dumps(page.__dict__, indent=4, ensure_ascii=False)
print(formatted_data)


{
    "page_content": "株主の権利および責任の追跡性\n– \n全取締役の年次選挙\n– \n無投票当選の取締役に対する多数決\n– \n臨時株主総会権\n– \nプロキシー・アクセス\n– \n株主のライツプランなし\n– \nスーパーマジョリティ条項なし\n– \n年間を通した強固な株主エンゲージメント・プロセス\n \n– 「コモンセンス原則2.0」への署名\n– \n投資家スチュワードシップ・グループ原則の支持\n– \n企業の目的に関する「Business Roundtable（ビジネス・ラウンド\nテーブル）」の声明への署名者\n– \n取締役を解任する株主の権利\n倫理的な影響\n↗\nコーポレート・ガバナンス\nIBMの取締役会は、取締役会の継続的な活力と卓越した職務執行\nを確保するために設計されたガバナンス原則を長年にわたって堅持\nしています。 25年以上にわたり、取締役会はこうした原則を反映した\n一連のガバナンス指針を定めてきました。これには、取締役会の過半\n数を独立系の役員とする方針、取締役と株主の利害を一致させるた\nめの株式報酬の重要性、そして経営陣以外の取締役による、経営陣\nを含まない幹部会議の定期的な開催などが含まれます。 「IBM取締\n役会のコーポレート・ガバナンスのガイドライン」には、企業のコーポ\nレート・ガバナンス事項に関するIBMの原則が反映されています。\nIBMの「ビジネス・コンダクト・ガイドライン」は、取締役、執行役員、社\n員の倫理規範です。 当社の取締役または執行役員のいずれかに適\n用される「ビジネス・コンダクト・ガイドライン」の修正または権利放棄\nは、取締役会または取締役会委員会のみが実行でき、IBMのWebサ\nイトで開示されます。 IBMにはまた、株主およびその他の利害関係者\nが取締役会または経営陣以外の取締役とコミュニケーションを取る\nことができるプロセスがあります。\nガバナンスの特記事項 \n効果的なリーダーシップ、独立した監督、強力なガバナンス\n– \n明確に定義された、強固な責任を有する独立したリード・ディレ\nクター\n– \n各取締役会における、独立したリード・ディレクターが主導する\n幹部会議の実施\n– \n多様性とスキル・経験の最適な組み合わせを重

In [45]:
# ページ内容
result = page.page_content

# ページ内容を見やすくするため、50文字ずつに改行して表示
line_length = 50
for i in range(0, len(result), line_length):
    print(result[i:i+line_length])

株主の権利および責任の追跡性
– 
全取締役の年次選挙
– 
無投票当選の取締役に対する多数決
– 

臨時株主総会権
– 
プロキシー・アクセス
– 
株主のライツプランなし
– 
スーパーマジョリテ
ィ条項なし
– 
年間を通した強固な株主エンゲージメント・プロセス
 
– 「コモンセンス原則2.0
」への署名
– 
投資家スチュワードシップ・グループ原則の支持
– 
企業の目的に関する「Busin
ess Roundtable（ビジネス・ラウンド
テーブル）」の声明への署名者
– 
取締役を解任す
る株主の権利
倫理的な影響
↗
コーポレート・ガバナンス
IBMの取締役会は、取締役会の継続的な活力
と卓越した職務執行
を確保するために設計されたガバナンス原則を長年にわたって堅持
しています。 25
年以上にわたり、取締役会はこうした原則を反映した
一連のガバナンス指針を定めてきました。これには、取
締役会の過半
数を独立系の役員とする方針、取締役と株主の利害を一致させるた
めの株式報酬の重要性、そ
して経営陣以外の取締役による、経営陣
を含まない幹部会議の定期的な開催などが含まれます。 「IBM取
締
役会のコーポレート・ガバナンスのガイドライン」には、企業のコーポ
レート・ガバナンス事項に関する
IBMの原則が反映されています。
IBMの「ビジネス・コンダクト・ガイドライン」は、取締役、執行役員
、社
員の倫理規範です。 当社の取締役または執行役員のいずれかに適
用される「ビジネス・コンダクト・
ガイドライン」の修正または権利放棄
は、取締役会または取締役会委員会のみが実行でき、IBMのWebサ

イトで開示されます。 IBMにはまた、株主およびその他の利害関係者
が取締役会または経営陣以外の取
締役とコミュニケーションを取る
ことができるプロセスがあります。
ガバナンスの特記事項 
効果的なリ
ーダーシップ、独立した監督、強力なガバナンス
– 
明確に定義された、強固な責任を有する独立したリー
ド・ディレ
クター
– 
各取締役会における、独立したリード・ディレクターが主導する
幹部会議の実施

– 
多様性とスキル・経験の最適な組み合わせを重視した、取締役会
および委員会の積極的な刷新
– 

取締役会リーダーシップ体制の年次レビュー
– 
秘密投票
1

## 2.2. 埋め込みモデル（embedding）のインスタンス作成

埋め込みモデルには、Azure OpenAI Serviceを使います。

In [46]:
EMBE_DEPLOYMENT_NAME = os.getenv("EMBE_DEPLOYMENT_NAME")
embeddings = OpenAIEmbeddings(deployment=EMBE_DEPLOYMENT_NAME, chunk_size=1)

埋め込みモデルを使って、テキストをベクトル化してみます。

In [47]:
text = "これは埋め込みのテストです。"
embeddings.embed_query(text)

message='Request to OpenAI API' method=post path=https://openai-isez2023006.openai.azure.com/openai/deployments/embedding-ada-002-ise2023006/embeddings?api-version=2023-05-15
api_version=2023-05-15 data='{"input": ["\\u3053\\u308c\\u306f\\u57cb\\u3081\\u8fbc\\u307f\\u306e\\u30c6\\u30b9\\u30c8\\u3067\\u3059\\u3002"], "model": "text-embedding-ada-002", "encoding_format": "base64"}' message='Post details'
message='OpenAI API response' path=https://openai-isez2023006.openai.azure.com/openai/deployments/embedding-ada-002-ise2023006/embeddings?api-version=2023-05-15 processing_ms=29.6762 request_id=f7f775f9-d67f-4ff7-8717-08f1e1c24466 response_code=200
body='{\n  "object": "list",\n  "data": [\n    {\n      "object": "embedding",\n      "index": 0,\n      "embedding": "vjWWvBlRSruXNM26bKZKu8Zuj7zSpjc7EW8BvLal7Ly3iqe8pvyfvHMZjzp+pxc81/wWPNI0wry+UNu7iTUgvKTDJTypNZq7WN74O/0YJb3zUFE8HYuUuvpRILxw4JS8BMQTvULC57sdpyk8gosBvVb8rruWpXK8it8/PF5RPbyW/KK8OsQJvcnDHrt9GY289DUMu5ZuGLxaUo48huCQPAGm3jw5w7k3QY

[-0.01833617314696312,
 -0.003087109187617898,
 -0.001565593178384006,
 -0.0030921949073672295,
 -0.01750887557864189,
 0.002802301663905382,
 -0.007900015451014042,
 -0.028887610882520676,
 -0.02045188657939434,
 -0.01952965185046196,
 0.0010917618637904525,
 0.009256241843104362,
 0.009215555153787136,
 -0.0237068273127079,
 -0.006692974828183651,
 -0.009778388775885105,
 0.010117445141077042,
 -0.004706103820353746,
 0.0075948648154735565,
 -0.04030703380703926,
 0.012775647453963757,
 -0.0011332962894812226,
 -0.00978516973555088,
 -0.018173426389694214,
 -0.03607560694217682,
 -0.007072717882692814,
 0.010354784317314625,
 -0.03162718564271927,
 -0.005340139381587505,
 -0.01480998657643795,
 0.011711010709404945,
 -0.01155504398047924,
 -0.019895832985639572,
 -0.033634401857852936,
 -0.0024225583765655756,
 -0.017224067822098732,
 -0.0021394463256001472,
 -0.009303709492087364,
 0.017373252660036087,
 0.017685186117887497,
 0.027178766205906868,
 2.2144626200315543e-05,
 0.011568

## 2-3. 埋め込みモデルを使って、PDFファイルからインデックスを作成する

ベクトルデータベースには、FAISSを使います。

In [48]:
db = FAISS.from_documents(pages, embeddings)

message='Request to OpenAI API' method=post path=https://openai-isez2023006.openai.azure.com/openai/deployments/embedding-ada-002-ise2023006/embeddings?api-version=2023-05-15
api_version=2023-05-15 data='{"input": [[1600, 38, 44971, 89154, 84477, 720, 2366, 16, 198, 68838, 720, 58400]], "model": "text-embedding-ada-002", "encoding_format": "base64"}' message='Post details'
message='OpenAI API response' path=https://openai-isez2023006.openai.azure.com/openai/deployments/embedding-ada-002-ise2023006/embeddings?api-version=2023-05-15 processing_ms=28.165 request_id=07a9417a-4fd7-49e4-bfec-6c8c6ead671d response_code=200
body='{\n  "object": "list",\n  "data": [\n    {\n      "object": "embedding",\n      "index": 0,\n      "embedding": "yjekvG0jY72Y5fm7DQzovD9eErxYmcs8HgH5vNP4cDyCjqW7I4SbO+a/ArwKW0Y8ukITu1B8r7xFl8m8ISWSPNzQZTwh/AW9BUubPPDpDzxpt2i7mqDSOSY1vbthw1i797QTO/JxpTwkrac8lkuAvDKVxzlFbj08z3qSPH4Y9LsaMYK8GHapO90QGjw0/oe8UAGLvLZRvTqOQMg88pqxvG5jFz3hjvi7l2rVuvVLUzxjfjG6oyYvPbWtjLxfEjc8U

検索してみます。

In [49]:
query = "IBMは気候変動のリスクにどのように取り組んでいますか？"

docs = db.similarity_search(query)
for doc in docs:
    print("page", doc.metadata["page"], doc)

message='Request to OpenAI API' method=post path=https://openai-isez2023006.openai.azure.com/openai/deployments/embedding-ada-002-ise2023006/embeddings?api-version=2023-05-15
api_version=2023-05-15 data='{"input": ["IBM\\u306f\\u6c17\\u5019\\u5909\\u52d5\\u306e\\u30ea\\u30b9\\u30af\\u306b\\u3069\\u306e\\u3088\\u3046\\u306b\\u53d6\\u308a\\u7d44\\u3093\\u3067\\u3044\\u307e\\u3059\\u304b\\uff1f"], "model": "text-embedding-ada-002", "encoding_format": "base64"}' message='Post details'


page 39 page_content='IBMのマネージド・インフラストラクチャー・サービス事業\nの分社化\nIBMは2021年11月3日に、マネージド・インフラストラクチャー・\nサービス事業を分社化し、現在はKyndryl社という上場企業になって\nいます。 これに伴い、本レポートの環境情報には、当該事業の10カ月\n分のデータが含まれています。 当社が2022年の結果を公表する際\nには、これまでとは大きく異なる事業領域が対象となります。\nエネルギーと気候変動\nIBMは30年前から気候変動への対応を明確にコミットしていま\nす。 IBMは創設パートナーとして、1992年に米国環境保護庁\n（EPA）がオフィス機器の国際的省エネルギー制度「ENERGY \nSTARプログラム」 を立ち上げる支援をしました。 1994年にCO2\n排出量の開示を開始し、2001年に初めて事業用の再生可能電力を\n購入しました。 2007 年には、気候変動に関するIBMの方針を発表\nしています。これは、気候変動が深刻な問題であり、世界規模でタイ\nムリーかつ有意義な行動をとるべきであることが認識され、今日の\nように重要視されるずっと前のことです。\n2015年には、パリ協定への支持を表明し、2017年にはその支持を\n再確認しました。 また、2019年には、Climate Leadership Council\n（クライメート・リーダーシップ・カウンシル）の創設メンバーとなり、\n「炭素配当金」による炭素税を市民に還元するという超党派の計画\nを引き続き支持しています。\n最新の目標\nIBMは2021年、再生可能電力の使用に関する第３次目標、温室効\n果ガス（GHG）排出量削減に関する第５次目標、温室効果ガス排出\nネット・ゼロの達成目標、および省エネ、データセンターのエネル\nギー効率、エネルギー効率の高い製品設計、サプライヤー、お客様エ\nンゲージメントなどに関する目標を設定しました。 これら目標のうち\n9つは、気候変動に関係しています。 5 つの目標は以下にて、その他\nの目標は本レポートの製品のエネルギー効率とサプライヤーの環境\n目標に関するセクションにて説明しています。 \n–\nIBMが消費する全世界の電力の75％を2025年までに、90％を\n20

message='OpenAI API response' path=https://openai-isez2023006.openai.azure.com/openai/deployments/embedding-ada-002-ise2023006/embeddings?api-version=2023-05-15 processing_ms=28.6228 request_id=3a178575-91ac-477e-a0a2-c24643bcfba1 response_code=200
body='{\n  "object": "list",\n  "data": [\n    {\n      "object": "embedding",\n      "index": 0,\n      "embedding": "SXExu+M5DL17nhw8O8BcvKIiqbwuoTw8GkjBvCDwZzwk8Z+6DcKTuyCJ2rtXIgY7ltzcO81Do7vhPM+5+6h5OzJCpTw13xI7KGfhPAnzwLxvijW6mEdlPHAq5jvRGbS7kZtDvMoGBbwA3JY8M+JVvB6wBj3dXwC8RjtRPS46r7rL39i7lNhhO2M9qzth2eC76RpWu0nYPjyYpzQ8GD0IvdO2ITuQAlG7PZJyPAVSWDv8c9G7QC/gPOevTTvXk/A7Uuiqu4vBtzp5od88J8ewPEY70buWbpG8lwcEvfuhuzvnFlu87Ff0uwYdsDtjpLi8/wmBPJkSPTyCUXy8XjU1PO/to7zlRMW6zRh8vC7TITwAdQm4lm4RvDfjjTyRNLY8vIcVPKhjQrsESxo9fN59Ojh8gLyTmIC8UIRgOthXijwsaBk87FC2u6uZIrz5PXG8kmYbPAfvxTvDAdK80IBBPCIfCrzaMN47WftZO+t+ILxb/1Q8iB0MPQIc+LwaFtw7wwFSO3XEkDyPN3k7ToDlvE55Jzo86wM6cpXuvNOEPDt4miG9/3dMuvAmxzumX8c6csCVPGh3Br3bYsO7uIZdPL0gCLyB5vO8R9TDvPdknbzqTDs8faKXu7mx

検索結果はデフォルトで4ドキュメントを含みます。それぞれ、PDFファイルのメタデータを持っています。

In [50]:
def document_to_dict(document):
    return document.__dict__

formatted_data = json.dumps(docs[0], default=document_to_dict, indent=4, ensure_ascii=False)
print(formatted_data)

{
    "page_content": "IBMのマネージド・インフラストラクチャー・サービス事業\nの分社化\nIBMは2021年11月3日に、マネージド・インフラストラクチャー・\nサービス事業を分社化し、現在はKyndryl社という上場企業になって\nいます。 これに伴い、本レポートの環境情報には、当該事業の10カ月\n分のデータが含まれています。 当社が2022年の結果を公表する際\nには、これまでとは大きく異なる事業領域が対象となります。\nエネルギーと気候変動\nIBMは30年前から気候変動への対応を明確にコミットしていま\nす。 IBMは創設パートナーとして、1992年に米国環境保護庁\n（EPA）がオフィス機器の国際的省エネルギー制度「ENERGY \nSTARプログラム」 を立ち上げる支援をしました。 1994年にCO2\n排出量の開示を開始し、2001年に初めて事業用の再生可能電力を\n購入しました。 2007 年には、気候変動に関するIBMの方針を発表\nしています。これは、気候変動が深刻な問題であり、世界規模でタイ\nムリーかつ有意義な行動をとるべきであることが認識され、今日の\nように重要視されるずっと前のことです。\n2015年には、パリ協定への支持を表明し、2017年にはその支持を\n再確認しました。 また、2019年には、Climate Leadership Council\n（クライメート・リーダーシップ・カウンシル）の創設メンバーとなり、\n「炭素配当金」による炭素税を市民に還元するという超党派の計画\nを引き続き支持しています。\n最新の目標\nIBMは2021年、再生可能電力の使用に関する第３次目標、温室効\n果ガス（GHG）排出量削減に関する第５次目標、温室効果ガス排出\nネット・ゼロの達成目標、および省エネ、データセンターのエネル\nギー効率、エネルギー効率の高い製品設計、サプライヤー、お客様エ\nンゲージメントなどに関する目標を設定しました。 これら目標のうち\n9つは、気候変動に関係しています。 5 つの目標は以下にて、その他\nの目標は本レポートの製品のエネルギー効率とサプライヤーの環境\n目標に関するセクションにて説明しています。 \n–\nIBMが消費する全世界の電力の75％を2025年までに、90％を\n2

インデックスはローカルに保存することができます。

In [51]:
index_path = "./index/IBM_2021_ESG_Report"
db.save_local(index_path)

保存したインデックスをロードすることができます。

In [52]:
db = FAISS.load_local(index_path, embeddings)

# 3. Retrieval QAチェーンの作成

## 3.1. LLMのインスタンス作成を作成する

LLMにAzure OpenAI Serviceを使います。

In [53]:
CHAT_DEPLOYMENT_NAME = os.getenv("CHAT_DEPLOYMENT_NAME")

llm = AzureChatOpenAI(
    deployment_name=CHAT_DEPLOYMENT_NAME,
    temperature=0.5,)

## 3.2. Retrieval QAチェーンを作成する

retriever（インデックス内の文書や埋め込みを検索するためのメソッド）とLLMを使って、Retrieval QAチェーンを生成します。

In [54]:
retriever = db.as_retriever()

qa_chain = RetrievalQA.from_chain_type(
    llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    verbose=True)

# 4. QA実施

Retrieval QAチェーンを使って、QAします。

In [110]:
query = "IBMの温室効果ガス排出量削減目標を教えてください。"
response = qa_chain({"query": query})



> Entering new RetrievalQA chain...


message='Request to OpenAI API' method=post path=https://openai-isez2023006.openai.azure.com/openai/deployments/embedding-ada-002-ise2023006/embeddings?api-version=2023-05-15
api_version=2023-05-15 data='{"input": ["IBM\\u306e\\u6e29\\u5ba4\\u52b9\\u679c\\u30ac\\u30b9\\u6392\\u51fa\\u91cf\\u524a\\u6e1b\\u76ee\\u6a19\\u3092\\u6559\\u3048\\u3066\\u304f\\u3060\\u3055\\u3044\\u3002"], "model": "text-embedding-ada-002", "encoding_format": "base64"}' message='Post details'
message='OpenAI API response' path=https://openai-isez2023006.openai.azure.com/openai/deployments/embedding-ada-002-ise2023006/embeddings?api-version=2023-05-15 processing_ms=35.9284 request_id=6868a9d1-4e67-49d1-b96b-510a1a3e8c78 response_code=200
body='{\n  "object": "list",\n  "data": [\n    {\n      "object": "embedding",\n      "index": 0,\n      "embedding": "pr9BvK728bwnhtE71788vONNQbzNvXE8FRW/vL/dlrrYhdm81TBRu5OFYLtdaHQ7shJ9PETBl7ydhV+852hmPPS9bTy92X68kr/DPHJsCrwR92c8N2sqPH333LtAasO7sxUvvDuHtbtaFNI8E4ZTvC0wYrqdhV+8


> Finished chain.


message='OpenAI API response' path=https://openai-isez2023006.openai.azure.com/openai/deployments/turbo0301-isez2023006/chat/completions?api-version=2023-05-15 processing_ms=5259.0097 request_id=071b8287-377e-4ec2-87eb-8949a8b2b58f response_code=200
body='{"id":"chatcmpl-7gftn9BCi1WqXKpx76HdIGo5HDR78","object":"chat.completion","created":1690405191,"model":"gpt-35-turbo","choices":[{"index":0,"finish_reason":"stop","message":{"role":"assistant","content":"IBMは、2030年までに温室効果ガスの排出量のネットゼロ達成を目指しています。また、2025年までに2005年基準年のCO2排出量に対するCO2排出量の削減率を40%から上回る削減率を目指しています。具体的には、再生可能エネルギーの利用拡大や省エネルギーの取り組み、サプライチェーンにおける温室効果ガス排出量削減の加速など、様々な取り組みを進めています。"}}],"usage":{"completion_tokens":184,"prompt_tokens":6969,"total_tokens":7153}}\n' headers="{'Date': 'Wed, 26 Jul 2023 20:59:56 GMT', 'Content-Type': 'application/json', 'Content-Length': '771', 'Connection': 'keep-alive', 'Cache-Control': 'no-cache, must-revalidate', 'access-control-allow-origin': '*', 'apim-request-id': '576c47a4-2934-41e4-ac51-a93202463c30

回答を見やすく表示します。

In [111]:
# レスポンスの回答部分
result = response["result"]

# 回答を見やすくするため、50文字ずつに改行して表示
line_length = 50
for i in range(0, len(result), line_length):
    print(result[i:i+line_length])

IBMは、2030年までに温室効果ガスの排出量のネットゼロ達成を目指しています。また、2025年まで
に2005年基準年のCO2排出量に対するCO2排出量の削減率を40%から上回る削減率を目指しています
。具体的には、再生可能エネルギーの利用拡大や省エネルギーの取り組み、サプライチェーンにおける温室効果
ガス排出量削減の加速など、様々な取り組みを進めています。


レスポンスを確認してみます。

In [112]:
def document_to_dict(document):
    return document.__dict__

formatted_data = json.dumps(response, default=document_to_dict, indent=4, ensure_ascii=False)
print(formatted_data)

{
    "query": "IBMの温室効果ガス排出量削減目標を教えてください。",
    "result": "IBMは、2030年までに温室効果ガスの排出量のネットゼロ達成を目指しています。また、2025年までに2005年基準年のCO2排出量に対するCO2排出量の削減率を40%から上回る削減率を目指しています。具体的には、再生可能エネルギーの利用拡大や省エネルギーの取り組み、サプライチェーンにおける温室効果ガス排出量削減の加速など、様々な取り組みを進めています。",
    "source_documents": [
        {
            "page_content": "IBMは2022年6月に、温室効果ガス排出量の計\n算プロセスと基礎データに関する年次の外部限\n定的保証監査を完了し、新しい再生可能電力と温\n室効果ガス排出量削減目標に対する2021年末\nの進捗状況を公表する予定です。\n+\nこれらの目標はボランタリー基準である「温室効果ガス・プロ\nトコル」を参考に、スコープ1、スコープ2、およびサード・パー\nティー・コロケーション・データセンターでの電力消費に関連\nするスコープ3の排出量を対象とします。\n+\nまたIBMは、自社のソリューションを活用して環境に良い\n影響をもたらすお客様との取り組みや研究プロジェクトを、 \n2025年までに100件開始することをコミットしています。 例\nえば、IBM Researchは、実現可能な素材の発見を加速させ、\n新しい炭素除去ソリューションの開発を支援しています。\n–\n2021年から2025年にかけて、最低3,000の新規省エネルギー・\nプロジェクトを実施し、275,000メガワット時（MWh）のエネル\nギー消費量を回避する。\n–\nデータセンターの平均冷却効率を、2025年までに基準年である\n2019年比で20%向上させる。\nIBMの新しい2025年の温室効果ガス排出量削減目標は、\n「国連の\n気候変動に関する政府間パネル（IPCC）」の科学者が、地球の温暖化\nを産業革命以前の水準から摂氏1.5度までに抑えるために必要と指\n摘している削減率を上回る基準となっています。\n私たちのエネルギーと排出の目標およびレポーティングは、IBMが\n所有または